In [2]:
# Импорт необходимых библиотек
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Flatten, Dense, Reshape, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# Пути к данным
images_dir = 'C:/Users/ARTEM/Documents/GitHub/AI-Systems/6Laba/Nails_segmentation/images'
labels_dir = 'C:/Users/ARTEM/Documents/GitHub/AI-Systems/6Laba/Nails_segmentation/labels'

# Функция для загрузки изображений и масок
def load_images_and_labels(images_dir, labels_dir, img_size=(128, 128)):
    images = []
    labels = []
    
    for img_name in os.listdir(images_dir):
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, img_name)

        img = load_img(img_path, target_size=img_size)
        img = img_to_array(img) / 255.0  # Нормализуем изображения
        
        label = load_img(label_path, target_size=img_size, color_mode='grayscale')
        label = img_to_array(label) / 255.0  # Маски в диапазоне [0, 1]
        
        images.append(img)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    print(f"Loaded {len(images)} images with shape {images.shape}")
    print(f"Loaded {len(labels)} labels with shape {labels.shape}")
    
    return images, labels

# Загрузка данных
X, y = load_images_and_labels(images_dir, labels_dir)

# Проверим размер данных
print(X.shape, y.shape)


Loaded 52 images with shape (52, 128, 128, 3)
Loaded 52 labels with shape (52, 128, 128, 1)
(52, 128, 128, 3) (52, 128, 128, 1)


In [3]:
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверим размеры
print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)


Train shapes: (41, 128, 128, 3) (41, 128, 128, 1)
Test shapes: (11, 128, 128, 3) (11, 128, 128, 1)


In [10]:
# Определение модели U-Net
def build_unet(input_shape):
    inputs = Input(shape=input_shape)
    
    # Этап сжатия
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Боттлнек
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Этап расширения
    u1 = UpSampling2D((2, 2))(c3)
    u1 = concatenate([u1, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = concatenate([u2, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    # Выходной слой (с одним каналом)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Построение модели U-Net
unet_model = build_unet(input_shape=(128, 128, 3))
unet_model.summary()

# Обучение U-Net
print("Training U-Net model...")
unet_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_14[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 64, 64,    │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_5[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_16[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 32, 32,    │          0 │ conv2d_17[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_6[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_18[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_2     │ (None, 64, 64,    │          0 │ conv2d_19[0][0]   │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 64, 64,    │          0 │ up_sampling2d_2[… │
│ (Concatenate)       │ 384)              │            │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 64, 64,    │    442,496 │ concatenate_2[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_20[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_3     │ (None, 128, 128,  │          0 │ conv2d_21[0][0]   │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 128, 128,  │          0 │ up_sampling2d_3[… │
│ (Concatenate)       │ 192)              │            │ conv2d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 128, 128,  │    110,656 │ concatenate_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_22[0][0] 

 Total params: 1,883,137 (7.18 MB)

 Trainable params: 1,883,137 (7.18 MB)

 Non-trainable params: 0 (0.00 B)

Training U-Net model...
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.9452 - loss: 0.5986 - val_accuracy: 0.9462 - val_loss: 0.2958
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9482 - loss: 0.2987 - val_accuracy: 0.9462 - val_loss: 0.3288
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9479 - loss: 0.3046 - val_accuracy: 0.9462 - val_loss: 0.2565
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9507 - loss: 0.2485 - val_accuracy: 0.9462 - val_loss: 0.2453
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9504 - loss: 0.2264 - val_accuracy: 0.9462 - val_loss: 0.2488
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9516 - loss: 0.2096 - val_accuracy: 0.9462 - val_loss: 0.2337
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9535 - loss: 0.2055 - val_accuracy: 0.9462 - val_loss: 0.2206
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9528 - loss: 0.1966 - val_accuracy: 0.9462 - v

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam

# Определение улучшенной модели полносвязной нейронной сети для сегментации
def build_fully_connected_model(input_shape):
    model = Sequential()

    # Первый сверточный слой для извлечения признаков
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Второй сверточный слой для углубленной обработки
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Третий сверточный слой
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Преобразование в одномерный вектор
    model.add(Flatten())

    # Полносвязные слои
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))

    # Слой для восстановления пространственной размерности (с использованием деконволюции)
    model.add(Dense(128 * 128, activation='sigmoid'))  # Количество пикселей в изображении (128x128)
    model.add(Reshape((128, 128, 1)))  # Преобразуем выход обратно в изображение (128x128x1)

    # Компиляция модели
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Построение улучшенной модели полносвязной нейронной сети
fc_model = build_fully_connected_model(input_shape=(128, 128, 3))
fc_model.summary()

# Обучение полносвязной нейронной сети
print("Training Fully Connected model...")
fc_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16384)          │     2,113,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 128, 128, 1)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,628,544 (40.54 MB)

 Trainable params: 10,628,544 (40.54 MB)

 Non-trainable params: 0 (0.00 B)

Training Fully Connected model...
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 311ms/step - accuracy: 0.5443 - loss: 0.6888 - val_accuracy: 0.7924 - val_loss: 0.5584
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.8395 - loss: 0.4670 - val_accuracy: 0.9412 - val_loss: 0.3644
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - accuracy: 0.9505 - loss: 0.2699 - val_accuracy: 0.9455 - val_loss: 0.2448
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.9493 - loss: 0.2052 - val_accuracy: 0.9390 - val_loss: 0.2768
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.9499 - loss: 0.2333 - val_accuracy: 0.9442 - val_loss: 0.2474
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.9541 - loss: 0.1843 - val_accuracy: 0.9461 - val_loss: 0.2864
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.9521 - loss: 0.1947 - val_accuracy: 0.9462 - val_loss: 0.2427
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.9468 - loss: 0.1903 

In [9]:
# Визуализация результатов
def plot_results(model, X_test, y_test):
    preds = model.predict(X_test)
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(X_test[0])
    axes[0].set_title("Input Image")
    
    axes[1].imshow(y_test[0].squeeze(), cmap='gray')
    axes[1].set_title("True Mask")
    
    axes[2].imshow(preds[0].squeeze(), cmap='gray')
    axes[2].set_title("Predicted Mask")
    
    plt.show()

# Визуализация результатов для U-Net
plot_results(unet_model, X_test, y_test)

# Визуализация результатов для полносвязной модели
plot_results(fc_model, X_test, y_test)


Training U-Net model...
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.9525 - loss: 0.1688 - val_accuracy: 0.9462 - val_loss: 0.1677
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9503 - loss: 0.1623 - val_accuracy: 0.9462 - val_loss: 0.1768
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9498 - loss: 0.1829 - val_accuracy: 0.9462 - val_loss: 0.1842
Epoch 4/10
2/3 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - accuracy: 0.9486 - loss: 0.1744

KeyboardInterrupt: 